[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Isotr0py/SakuraLLM-Notebooks/blob/main/Sakura-13B-Galgame-Colab.ipynb)

In [ ]:
#@title 初始化环境
#@markdown 挂载Google网盘
Mount_GDrive = False # @param {type:"boolean"}
if Mount_GDrive:
  from google.colab import drive

  drive.mount('/content/gdrive')
  ROOT_PATH = "/content/gdrive/MyDrive"
else:
  ROOT_PATH = "/content"
!nvidia-smi

In [ ]:
#@title 安装依赖
%cd $ROOT_PATH
!git clone https://github.com/SakuraLLM/Sakura-13B-Galgame.git

%cd Sakura-13B-Galgame
LLAMA_CPP = True # @param {type:"boolean"}
VLLM = True # @param {type:"boolean"}
if LLAMA_CPP:
  !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
if VLLM:
  !pip install -q -U torch --index-url https://download.pytorch.org/whl/cu121
  !pip install vllm
!pip install -q -r requirements.txt
!pip install -q pyngrok

In [ ]:
#@title 翻译EPUB
MODEL = "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit" # @param ["SakuraLLM/Sakura-13B-LNovel-v0_8-3bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-8bit"]
EPUB_PATH = "novel.epub" # @param {type:"string"}
OUTPUT_FOLDER = "output/" # @param {type:"string"}

%cd $ROOT_PATH/Sakura-13B-Galgame
!python translate_epub.py \
    --model_name_or_path $MODEL \
    --trust_remote_code \
    --model_version 0.8 \
    --use_gptq_model \
    --text_length 512 \
    --data_path $EPUB_PATH \
    --output_folder $OUTPUT_FOLDER

In [ ]:
#@title 翻译文本文件
MODEL = "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit" # @param ["SakuraLLM/Sakura-13B-LNovel-v0_8-3bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-8bit"]
DATA_PATH = "novel.txt" # @param {type:"string"}
OUTPUT_PATH = "novel_translated.txt" # @param {type:"string"}

%cd $ROOT_PATH/Sakura-13B-Galgame
!python translate_novel.py \
    --model_name_or_path $MODEL \
    --trust_remote_code \
    --model_version 0.8 \
    --use_gptq_model \
    --text_length 512 \
    --data_path $DATA_PATH \
    --output_path $OUTPUT_PATH

In [ ]:
#@title 运行API后端(Transformers AutoGPTQ)
#@markdown 使用[ngrok](https://ngrok.com/)进行API映射
from huggingface_hub import snapshot_download
from pathlib import Path

ngrokToken = ""  # @param {type:"string"}
if ngrokToken:
  from pyngrok import conf, ngrok
  conf.get_default().auth_token = ngrokToken
  conf.get_default().monitor_thread = False
  ssh_tunnels = ngrok.get_tunnels(conf.get_default())
  if len(ssh_tunnels) == 0:
      ssh_tunnel = ngrok.connect(5000)
      print('address：'+ssh_tunnel.public_url)
  else:
      print('address：'+ssh_tunnels[0].public_url)
else:
  import subprocess
  import threading
  def start_localtunnel(port):
      p = subprocess.Popen(["npx", "localtunnel", "--port", f"{port}"], stdout=subprocess.PIPE)
      for line in p.stdout:
          print(line.decode(), end='')
  threading.Thread(target=start_localtunnel, daemon=True, args=(5000,)).start()


%cd $ROOT_PATH/Sakura-13B-Galgame
MODEL = "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit" # @param ["SakuraLLM/Sakura-13B-LNovel-v0_8-3bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-8bit"]
MODEL_PATH = f"models/{MODEL.split('/')[-1]}"
if not Path(MODEL_PATH).exists():
  snapshot_download(repo_id=MODEL,local_dir=MODEL_PATH,local_dir_use_symlinks=False)
  !sed -i '521c\        return nn.functional.linear(hidden_states, norm_weight.to(hidden_states.dtype))' $MODEL_PATH/modeling_baichuan.py

!python server.py \
  --model_name_or_path $MODEL_PATH \
  --use_gptq_model \
  --model_version 0.8 \
  --trust_remote_code \
  --no-auth

In [ ]:
#@title 运行API后端(vLLM)
#@markdown 使用[ngrok](https://ngrok.com/)进行API映射（ngrokToken留空则使用[localtunnel](https://github.com/localtunnel/localtunnel)）
from huggingface_hub import snapshot_download
from pathlib import Path

ngrokToken = ""  # @param {type:"string"}
if ngrokToken:
  from pyngrok import conf, ngrok
  conf.get_default().auth_token = ngrokToken
  conf.get_default().monitor_thread = False
  ssh_tunnels = ngrok.get_tunnels(conf.get_default())
  if len(ssh_tunnels) == 0:
      ssh_tunnel = ngrok.connect(5000)
      print('address：'+ssh_tunnel.public_url)
  else:
      print('address：'+ssh_tunnels[0].public_url)
else:
  import subprocess
  import threading
  def start_localtunnel(port):
      p = subprocess.Popen(["npx", "localtunnel", "--port", f"{port}"], stdout=subprocess.PIPE)
      for line in p.stdout:
          print(line.decode(), end='')
  threading.Thread(target=start_localtunnel, daemon=True, args=(5000,)).start()


%cd $ROOT_PATH/Sakura-13B-Galgame
MODEL = "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit" # @param ["SakuraLLM/Sakura-13B-LNovel-v0_8-4bit"]

!RAY_memory_monitor_refresh_ms="0" python server.py \
  --model_name_or_path $MODEL \
  --vllm \
  --use_gptq_model \
  --model_version 0.8 \
  --trust_remote_code \
  --no-auth \
  --enforce_eager \
  --gpu_memory_utilization 0.95

In [ ]:
#@title 运行API后端(llama-cpp-python)
#@markdown 使用[ngrok](https://ngrok.com/)进行API映射（ngrokToken留空则使用[localtunnel](https://github.com/localtunnel/localtunnel)）
from huggingface_hub import hf_hub_download
from pathlib import Path

ngrokToken = ""  # @param {type:"string"}
if ngrokToken:
  from pyngrok import conf, ngrok
  conf.get_default().auth_token = ngrokToken
  conf.get_default().monitor_thread = False
  ssh_tunnels = ngrok.get_tunnels(conf.get_default())
  if len(ssh_tunnels) == 0:
      ssh_tunnel = ngrok.connect(5000)
      print('address：'+ssh_tunnel.public_url)
  else:
      print('address：'+ssh_tunnels[0].public_url)
else:
  import subprocess
  import threading
  def start_localtunnel(port):
      p = subprocess.Popen(["npx", "localtunnel", "--port", f"{port}"], stdout=subprocess.PIPE)
      for line in p.stdout:
          print(line.decode(), end='')
  threading.Thread(target=start_localtunnel, daemon=True, args=(5000,)).start()


%cd $ROOT_PATH/Sakura-13B-Galgame
MODEL = "sakura-13b-lnovel-v0.9-Q4KM_awq4bit" # @param ["sakura-13b-lnovel-v0.9-Q8_0", "sakura-13b-lnovel-v0.9-Q6_K", "sakura-13b-lnovel-v0.9-Q5_K_M", "sakura-13b-lnovel-v0.9-Q4KM_awq4bit", "sakura-13b-lnovel-v0.9-Q4_K_M", "sakura-13b-lnovel-v0.9-Q3_K_M", "sakura-13b-lnovel-v0.9-Q2_K"]
MODEL_PATH = f"./models/{MODEL}.gguf"
if not Path(MODEL_PATH).exists():
    hf_hub_download(repo_id="SakuraLLM/Sakura-13B-LNovel-v0.9b-GGUF", filename=f"{MODEL}.gguf", local_dir="models/")

!python server.py \
  --model_name_or_path $MODEL_PATH \
  --llama_cpp \
  --use_gpu \
  --model_version 0.9 \
  --trust_remote_code \
  --no-auth